In [1]:
from __future__ import print_function
import cPickle as pickle
import csv
from datetime import datetime

import numpy as np
import tensorflow as tf

from keras import backend as K
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences

from data_generator import DataGenerator
from lstm_cvae_model import ModelConfig

def get_sample_config():
    sample_config = {
        "model_dir": "/Users/tongwang/Playground/deepjoke/code/model_checkpoints/lstm_cvae/20170618_072219",
        "starter_sentences": ["a sexy", "what", "why", "i have a dream", "once upon a time", "trump"],
        "temperatures": [None, 0.2, 0.5, 1.0, 1.5],
        "scores": [0, 1, 5, 10, 20],
        "num": 5
    }
    return sample_config

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def sample(preds, temperature=None):
    """Helper function to sample an index from a probability array; if temperature is None, 
    then sample greedily"""
    if temperature is None:
        return np.argmax(preds)
    else:
        preds = np.asarray(preds).astype('float64')
        preds = softmax(preds)  # Convert logits into probabilities
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def tokens_to_words(tokens, tokenizer, eos=""):
    """Helper function to turn an 1-d array of tokens tokenized by tokenizer back to words"""
    reverse_word_index = {index: word for word, index in tokenizer.word_index.iteritems()}
    reverse_word_index[0] = eos
    words = [reverse_word_index.get(token) for token in tokens]
    text = " ".join(words)
    return text

Using TensorFlow backend.


In [2]:
model_dir = "/Users/tongwang/Playground/deepjoke/code/model_checkpoints/lstm_cvae/20170618_072219"
encoder_path = model_dir + "/encoder_checkpoint"
generator_path = model_dir + "/generator_checkpoint"
tokenizer_path = model_dir + "/tokenizer.p"
model_config_path = model_dir + "/model_config.p"

encoder = load_model(encoder_path)
generator = load_model(generator_path)
tokenizer = pickle.load(open(tokenizer_path, "r"))
model_config = pickle.load(open(model_config_path, "r"))

//anaconda/lib/python2.7/site-packages/keras/models.py:258: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [29]:
def generate_text(target_score, generator, model_config, tokenizer, 
                  starter_sentence="", temperature=None, variation=1.0, eos=""):
    """Function to generate paragraphs given a target score, a random latent vector,
    and (optionally) a starter sentence.
    
    Args:
        -target_score
        -generator
        -model_config
        -tokenizer
        -temperature: if None, generate text greedily; otherwise sample stochastically
    Returns:
        -model_config.batch_size many pieces of text
    """
    # Prepare inputs
    z = np.random.normal(scale=variation., size=(model_config.batch_size, model_config.latent_size))
    print(z)
    scores = np.repeat(target_score, model_config.batch_size)
    cur_sentence = [starter_sentence]
    cur_sequence = tokenizer.texts_to_sequences(cur_sentence)
    cur_sequence = pad_sequences(cur_sequence, maxlen=model_config.max_sequence_length,
                                 padding='post', truncating='post')
    cur_sequence = np.repeat(cur_sequence, model_config.batch_size, axis=0)
    
    reverse_word_index = {index: word for word, index in tokenizer.word_index.iteritems()}

    # Iteratively predict the next word
    """
    while True:
        true_len = len(cur_sequence[0][cur_sequence[0]>0])
        if true_len == model_config.max_sequence_length:
            break
        next_preds = generator.predict([cur_sequence, scores, z])[0, true_len-1, :] # predicted next word
        next_token = sample(next_preds, temperature)
        if next_token == 0:
            break
        cur_sequence[0][true_len] = next_token"""
    while True:
        true_lens = np.sum(cur_sequence > 0, axis=1)
        last_tokens = np.array([cur_sequence[i, true_lens[i]-1] for i in range(model_config.batch_size)])
        print(true_lens)
        print(last_tokens)
        if np.min(np.logical_or(
            (true_lens == model_config.max_sequence_length),
            (last_tokens == 0))) > 0:
            break
        all_preds = generator.predict([cur_sequence, scores, z])
        for i in range(model_config.batch_size):
            if last_tokens[i] == 0:
                continue
            else:
                next_preds = all_preds[i, true_lens[i]-1, :]
                next_token = sample(next_preds, temperature)
                cur_sequence[i, true_lens[i]] = next_token

    pred_texts = []
    for i in range(model_config.batch_size):
        pred_sequence = cur_sequence[i][cur_sequence[i]>0]
        pred_text = tokens_to_words(pred_sequence, tokenizer=tokenizer, eos=eos)
        pred_texts.append(pred_text)
    
    return pred_texts

In [30]:
s = generate_text(generator=generator, model_config=model_config, tokenizer=tokenizer,
              target_score=2, starter_sentence="a", variation=3.0, temperature=0.2)

[[ -9.15379304   8.27983002   4.92072988 ...,  -1.42404489 -16.31653623
   -6.78862419]
 [ 11.10850578   4.5269289    3.31255394 ..., -18.58796288 -11.939461
   -6.2921765 ]
 [ -4.67132108   4.68724232   8.91312393 ...,  -1.34071219 -14.39342259
    4.81227496]
 ..., 
 [-19.62167796  -7.77455219  -8.27733414 ...,  16.73833075   8.09323821
  -17.93385031]
 [  6.18136546   7.4858186   -7.84384357 ..., -16.35000095  -3.1640482
   -5.57649955]
 [  7.83658104  -2.37994521 -13.66690602 ...,  -7.76565026   4.85707867
    1.38275973]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[  23    4   23   23 1904 1309    8    1 1311   23  101   23   23   23   23
  101 1856   23    4 1856   23   23   23   23   23   23    1   23  171   23
   23   23]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[ 108 1169   13    5 3681 2276  575 185

KeyboardInterrupt: 

In [9]:
type(s)

numpy.ndarray

In [20]:
a = np.random.normal(size=(5, 4))

In [21]:
a

array([[ 0.68480836,  0.34998149,  0.38515765, -0.41151523],
       [ 1.52700636,  0.66575251,  0.31049105,  0.18926559],
       [ 0.93849434,  0.55826684, -0.61776509,  0.21317279],
       [-0.37012515, -1.25836485,  0.45113172,  0.88775646],
       [-0.03215376, -0.3352601 ,  0.82262767, -0.28751208]])